# iBioFoundry - Plate Prep - OT2
author: Camillo Moschner | version: 1.0 | date: 24.09.2022 | license = 

# 1- Metadata

In [1]:
metadata = {
    'protocolName': 'iBF_plate_prep',
    'author': 'Camillo Moschner <cm967@cam.ac.uk> / <camillo.moschner@gmail.com>',
    'description': 'Automated preparation of cPCR reactions',
    'apiLevel': '2.12',
    'Date': '24.09.2022',
    'pipette_configuration':{'left':'p20_single_gen2',
                             'right':'p300_single_gen2'}
    }

# 2- Import Statements

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display, clear_output, Audio, display, display_html
from datetime import date
import os
from itertools import product

from iBioFoundry_helper import *
from opentrons import protocol_api, execute, simulate

## Plate Pipetting Master Inputs
The following cell requires changing between experimental setups (and, for simplicity for the user, is supposed to be the only part of the protocol that requires any modification).

In [3]:
# establish whether you are performing a simulation (e.g. at the design stage) or are executing the script on the OT-2
script_mode = 'simulation' # 'simulation' or 'execution'
experiment_name = 'JUMP_LVL2_DVs_PR_test'
audiofeedback = False # True if you want to to receive audio-notification of a finished process, else change to Falsev

# resuspended colony info
resusp_colony_plate_format = 'opentrons_96_aluminumblock_generic_pcr_strip_200ul' # 'opentrons_96_aluminumblock_biorad_wellplate_200ul'
resusp_col_plate_column_usage = 'odd' # allows you to selectively only use 'odd'- or 'even'-numbered columns or 'all'; useful for 0.2 ml tube usage
# extra reagent info
reagents_rack_format  = 'opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap'
# culture plate info
culture_plate_format = 'corning_96_wellplate_360ul_flat'
culture_plate_column_usage = 'all'
# final reaction composition
cell_culture_vol = 200.0 # units: µl
resusp_col_for_cell_culture_vol = 1.0 # units: µl

In [4]:
# establish sole Jupyter Notebook control 
if script_mode == 'execution':
    try:
        os.system("systemctl stop opentrons-robot-server")
        protocol = execute.get_protocol_api(metadata['apiLevel']);
    except:
        protocol = execute.get_protocol_api(metadata['apiLevel']);
    print("You are operating this script in exectution mode!\nYou should see the lights turn on and hear the gantry homing!")
    startNow()
elif script_mode == 'simulation':
    protocol = simulate.get_protocol_api(metadata['apiLevel']);
    print("You are operating this script in simulation mode!")
protocol.set_rail_lights(True)
protocol.home()

/Users/camillomoschner/.opentrons/robot_settings.json not found. Loading defaults
/Users/camillomoschner/.opentrons/deck_calibration.json not found. Loading defaults


You are operating this script in simulation mode!


# 3- Function Definitions

In [5]:
def smart_transfer_liquid(transfer_vol, aspiration_pos,dispensation_pos,
                          pick_up_tip=True, touch_tip_before=True,
                          cyclical_mix_before=None,mix_before_reps=2,mix_rate=0.5,
                          asp_bot_clearance=1.5,dispens_bot_clearance=1.5,
                          asp_rate=0.8,disp_rate=0.5,
                          cyclical_mix_after=None,mix_after=None,
                          dispens_top_offset=-2, blow_out=False, air_gap=False, touch_tip=True, touch_miniscus=None,
                          drop_tip=True):
    """Liquid transfer that automatically identifies which tip to use.
    """
    if (transfer_vol >= p20_left.min_volume) & (transfer_vol <= p20_left.max_volume):
        pipette_to_use = p20_left
    elif (transfer_vol > p300_right.min_volume):
        pipette_to_use = p300_right
    else:
        print(f"WARNING: transfer volume not manageable")
    if pick_up_tip==True:
        pipette_to_use.pick_up_tip()
    if cyclical_mix_before != None: # give tuple (repetitions, mix_volume, well_bottom_clearance)
        for i in np.arange(cyclical_mix_before[0]):
            pipette_to_use.aspirate(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[2]))
            pipette_to_use.dispense(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[3]))
    if mix_before_reps != 0:
        pipette_to_use.mix(mix_before_reps,transfer_vol*0.8,aspiration_pos.bottom(asp_bot_clearance),mix_rate)
    # aspiration
    pipette_to_use.aspirate(transfer_vol,aspiration_pos.bottom(asp_bot_clearance), asp_rate)
    pipette_to_use.move_to(aspiration_pos.top())
    if air_gap != False:
        pipette_to_use.air_gap(air_gap)
    protocol.delay(seconds=1)
    if touch_tip_before == True:
        pipette_to_use.touch_tip(v_offset=-2, speed=150)
    # dispensation
    pipette_to_use.dispense(transfer_vol,dispensation_pos.bottom(dispens_bot_clearance),disp_rate)
    if cyclical_mix_after != None: # give tuple (repetitions, mix_volume, asp- , disp-well_bottom_clearance)
        for i in np.arange(cyclical_mix_after[0]):
            pipette_to_use.aspirate(cyclical_mix_after[1],dispensation_pos.bottom(cyclical_mix_after[2]))
            pipette_to_use.dispense(cyclical_mix_after[1],dispensation_pos.bottom(cyclical_mix_after[3]))
    if mix_after != None:
        pipette_to_use.mix(mix_after[0],mix_after[1], dispensation_pos.bottom(mix_after[2]),mix_rate)
    pipette_to_use.move_to(dispensation_pos.top())
    protocol.delay(seconds=1)
    if blow_out == True:
        pipette_to_use.blow_out(dispensation_pos.top(dispens_top_offset))
    if touch_tip == True:
        pipette_to_use.touch_tip(v_offset=dispens_top_offset, speed=150)
    if touch_miniscus != None:
        protocol.max_speeds['Z'] = 10  # limit z axis to 50 mm/s to avoid solution adhereing to outer tip surface
        pipette_to_use.move_to(dispensation_pos.bottom(touch_miniscus))
        protocol.max_speeds['Z'] = None # reset z axis speed
    if drop_tip==True:
        pipette_to_use.drop_tip()
        
def smart_distribute_liquid(single_distr_vol, aspiration_pos,dispensation_pos_list,
                            asp_bot_clearance=1.1,dispens_bot_clearance=1.8,
                            asp_flow_rate = 1,disp_flow_rate = 1, new_disposal_volume=None,
                            touch_tip_bool=True,blow_out_bool=False):
    """
    Liquid distribution from one aspiration well into multiple destination wells that automatically identifies which pipette to use.
    """
    if (single_distr_vol >= p20_left.min_volume) & (single_distr_vol <= p20_left.max_volume):
        pipette_to_use = p20_left
    elif (single_distr_vol > p300_right.min_volume):
        pipette_to_use = p300_right
    # save original/default flow rates
    original_asp_flow_rate = pipette_to_use.flow_rate.aspirate
    original_disp_flow_rate = pipette_to_use.flow_rate.dispense
    # change flow rates if desired
    pipette_to_use.flow_rate.aspirate = original_asp_flow_rate *asp_flow_rate
    pipette_to_use.flow_rate.dispense = original_disp_flow_rate *disp_flow_rate
    # change well_bottom_clearances if desired
    pipette_to_use.well_bottom_clearance.aspirate = asp_bot_clearance
    pipette_to_use.well_bottom_clearance.dispense = dispens_bot_clearance
    # execute
    if new_disposal_volume != None:
        pipette_to_use.distribute(single_distr_vol, aspiration_pos,dispensation_pos_list, 
                                  touch_tip=touch_tip_bool,disposal_volume=new_disposal_volume,blow_out=blow_out_bool)
    else:
        pipette_to_use.distribute(single_distr_vol, aspiration_pos,dispensation_pos_list,
                                  touch_tip=touch_tip_bool,blow_out=blow_out_bool)
    # set well_bottom_clearances back to default
    pipette_to_use.well_bottom_clearance.dispense = 1
    pipette_to_use.well_bottom_clearance.aspirate = 1
    # set flow rates back to default
    pipette_to_use.flow_rate.aspirate = original_asp_flow_rate
    pipette_to_use.flow_rate.dispense = original_disp_flow_rate

def flatten(l):
    return [item for sublist in l for item in sublist]

---
# 4- Deck assignments

In [6]:
"""Deck creation"""
deck_slot_list =list(np.arange(1,12))
deck_slot_list.append('bin')
deck_slot_df = pd.DataFrame( np.flip(np.array(deck_slot_list).reshape(4,3), axis=0) )

## i. Labware
Resource: [Opentrons Labware Library Webpage](https://labware.opentrons.com/?_gl=1*1iyc1t4*_ga*MTU1MTM1NzU5MS4xNjE2OTczMzY5*_ga_GNSMNLW4RY*MTYzMDYxNzY1NS4yMC4wLjE2MzA2MTc2NTUuMA..&_ga=2.110416899.767557364.1630617656-1551357591.1616973369)
### a. Modules

In [7]:
temp_mod_name  = 'temperature module gen2'
temp_mod = protocol.load_module(temp_mod_name,'3')
deck_slot_df.iloc[3,2] = temp_mod_name

### b. Racks & Plates

In [8]:
tiprack20a = protocol.load_labware('opentrons_96_tiprack_20ul','8')
tiprack20b = protocol.load_labware('opentrons_96_tiprack_20ul','11')
tiprack300a = protocol.load_labware('opentrons_96_tiprack_300ul','10')
# tuberacks & plates
resusp_colony_plate = protocol.load_labware(resusp_colony_plate_format,'2') # opentrons_96_aluminumblock_biorad_wellplate_200ul
culture_plate = temp_mod.load_labware(culture_plate_format,'3')
reagents_rack = protocol.load_labware(reagents_rack_format,'1') #  biorad_96_wellplate_200ul_pcr
large_vol_rack = protocol.load_labware('opentrons_6_tuberack_falcon_50ml_conical','4') 

In [9]:
labware_list = [tiprack20a,tiprack20b, tiprack300a,
                resusp_colony_plate,culture_plate, reagents_rack,large_vol_rack]
_ =[update_labware(deck_slot_df, labware_item) for labware_item in labware_list]
# visually inspect deck
print("\nFinal OT-2 Deck Configuration:");(deck_slot_df)


Final OT-2 Deck Configuration:


,0,1,2
0,10-opentrons_96_tiprack_20ul,11-opentrons_96_tiprack_20ul,bin
1,7-opentrons_96_tiprack_300ul,8,9
2,4-opentrons_6_tuberack_falcon_50ml_conical,5,6
3,1-opentrons_24_tuberack_eppendorf_1.5ml_safelo...,2-opentrons_96_aluminumblock_generic_pcr_strip...,temperature module gen2


## ii. Pipettes

Resource: [Opentrons Pipettes API Webpage](https://docs.opentrons.com/v2/new_pipette.html)

In [10]:
p20_left = protocol.load_instrument('p20_single_gen2','left', tip_racks=[tiprack20a,tiprack20b])
p300_right = protocol.load_instrument('p300_single_gen2','right', tip_racks=[tiprack300a])

### In-Notebook Calibration

In [11]:
#if in_jupyter_calibration == True:
#import opentrons
pipette_to_calibrate = p20_left # p300_right
labware_to_calibrate_to = resusp_colony_plate # resusp_colony_plate | culture_plate | reagents_rack | large_vol_rack
labware_position_to_test = 'A1'

In [12]:
# step 0: pick up tip for the pipette you want to calibrate and move to the labware to be calibrated
pipette_to_calibrate.pick_up_tip()
pipette_to_calibrate.move_to(labware_to_calibrate_to[labware_position_to_test].top())

<InstrumentContext: p20_single_v2.0 in LEFT>

In [13]:
# step 1: set_offset based on identifying in which direction the tip needs to move
# (can be an iderative process as this cell immediately moves to the new position)
labware_to_calibrate_to.set_offset(x=(0.),y=(0.),z=(0.))
pipette_to_calibrate.move_to(labware_to_calibrate_to[labware_position_to_test].top())

<InstrumentContext: p20_single_v2.0 in LEFT>

In [14]:
# step 2: return tip (if you haven'crashed it
pipette_to_calibrate.return_tip()

<InstrumentContext: p20_single_v2.0 in LEFT>

---
# 5- LH Step Calculations 
## Design Allocation

Define your experimental conditions:

In [15]:
# constructs
samples_cell_levels = ['y']
construct_levels = ['LVL2_NTC', 'pJUMP47-2A(pSC101)', 'pJUMP43-2A(pBBR1)', 'pJUMP46-2A(p15A)', 
                    'pJUMP44-2A(pRO1600-ColE1)', 'pJUMP48-2A(pUC)']
# hosts
host = ['DH5a']
# environments
media = ['EZRDM']
abx_c_levels = ['0ug', '25ug', '50ug', 
                '75ug', '100ug'] # Streptomycin; units: ug/ml
techn_replicates_no = 3

Test out how many combinations of runs will be generated using different controls, replicate numbers, and testing conditions:

In [33]:
NTC = [[('n', 'nan', 'NTC', 'EZRDM', abx_c_levels[0])]*techn_replicates_no]  + [[('y', 'DH5a','just_cells', 'EZRDM', abx_c_levels[0])]*techn_replicates_no] #[[('y', 'DH5a', abx_c_levels[0])]*techn_replicates_no] #
samples = [[x]*techn_replicates_no for x in list(product(samples_cell_levels, host, construct_levels, media, abx_c_levels))]
complete_run_list = flatten(NTC + samples)
complete_run_df = pd.DataFrame( complete_run_list, columns=['cells', 'host', 'samples', 'media', 'abx_c'])
complete_run_df['replicate']  = (len(NTC)+len(samples))*list(range(techn_replicates_no))
complete_run_df['ID'] = complete_run_df.samples+complete_run_df.abx_c.apply(lambda x: f"|{x}")+complete_run_df.replicate.apply(lambda x: f"|{x}")
complete_run_df#.head(50)

,cells,host,samples,media,abx_c,replicate,ID
0,n,nan,NTC,EZRDM,0ug,0,NTC|0ug|0
1,n,nan,NTC,EZRDM,0ug,1,NTC|0ug|1
2,n,nan,NTC,EZRDM,0ug,2,NTC|0ug|2
3,y,DH5a,just_cells,EZRDM,0ug,0,just_cells|0ug|0
4,y,DH5a,just_cells,EZRDM,0ug,1,just_cells|0ug|1
...,...,...,...,...,...,...,...
91,y,DH5a,pJUMP48-2A(pUC),EZRDM,75ug,1,pJUMP48-2A(pUC)|75ug|1
92,y,DH5a,pJUMP48-2A(pUC),EZRDM,75ug,2,pJUMP48-2A(pUC)|75ug|2
93,y,DH5a,pJUMP48-2A(pUC),EZRDM,100ug,0,pJUMP48-2A(pUC)|100ug|0
94,y,DH5a,pJUMP48-2A(pUC),EZRDM,100ug,1,pJUMP48-2A(pUC)|100ug|1


## Object Creation

In [34]:
# create overnight cell culture 8-strip object
on_cultures_rack_obj = Plate(96, 'O/N cultures aliquoted in 8-strip tubes')
on_cultures_rack_obj.layout = extend_fill_plate_df_with_list(on_cultures_rack_obj.layout, list(complete_run_df.samples.unique()),
                                                             fill_start_position='A1', fill_first='columns')
# create media rack object
media_rack_obj = Plate(6, 'media in 50ml Falkon tubes')
media_rack_obj.layout = extend_fill_plate_df_with_list(media_rack_obj.layout, abx_c_levels,
                                                       fill_start_position='A1', fill_first='columns')
# create cell culture media plate object
culture_plate_media_obj = Plate(96, 'cell culture plate (abx concentrations)')
culture_plate_media_obj.layout = extend_fill_plate_df_with_list(culture_plate_media_obj.layout, complete_run_df.abx_c.to_list(),
                                                                fill_start_position='A1', fill_first='columns')
# create combined cell culture plate object
culture_plate_obj = Plate(96, 'cell culture plate (ID)')
culture_plate_obj.layout = extend_fill_plate_df_with_list(culture_plate_obj.layout, complete_run_df.ID.to_list(),
                                                          fill_start_position='A1', fill_first='columns')

In [35]:
complete_run_df['well'] = culture_plate_obj.occupied_wells.keys()
complete_run_df['media_source'] = complete_run_df.abx_c.apply(lambda x: large_vol_rack[find_df_coordinates(media_rack_obj.layout, x)])
complete_run_df['sample_source'] = complete_run_df.samples.apply(lambda x: resusp_colony_plate[find_df_coordinates(on_cultures_rack_obj.layout, x)])
complete_run_df['destination'] = complete_run_df.well.apply(lambda x: culture_plate[x])
complete_run_df['prep_date'] = [str(date.today()) for x in range(len(complete_run_df))]
complete_run_df.set_index('well',inplace=True)
complete_run_df

,cells,host,samples,media,abx_c,replicate,ID,media_source,sample_source,destination,prep_date
well,,,,,,,,,,,
A1,n,nan,NTC,EZRDM,0ug,0,NTC|0ug|0,A1 of Opentrons 6 Tube Rack with Falcon 50 mL ...,A1 of Opentrons 96 Well Aluminum Block with Ge...,A1 of 3 on Temperature Module GEN2 on 3,2022-09-24
B1,n,nan,NTC,EZRDM,0ug,1,NTC|0ug|1,A1 of Opentrons 6 Tube Rack with Falcon 50 mL ...,A1 of Opentrons 96 Well Aluminum Block with Ge...,B1 of 3 on Temperature Module GEN2 on 3,2022-09-24
C1,n,nan,NTC,EZRDM,0ug,2,NTC|0ug|2,A1 of Opentrons 6 Tube Rack with Falcon 50 mL ...,A1 of Opentrons 96 Well Aluminum Block with Ge...,C1 of 3 on Temperature Module GEN2 on 3,2022-09-24
D1,y,DH5a,just_cells,EZRDM,0ug,0,just_cells|0ug|0,A1 of Opentrons 6 Tube Rack with Falcon 50 mL ...,B1 of Opentrons 96 Well Aluminum Block with Ge...,D1 of 3 on Temperature Module GEN2 on 3,2022-09-24
E1,y,DH5a,just_cells,EZRDM,0ug,1,just_cells|0ug|1,A1 of Opentrons 6 Tube Rack with Falcon 50 mL ...,B1 of Opentrons 96 Well Aluminum Block with Ge...,E1 of 3 on Temperature Module GEN2 on 3,2022-09-24
...,...,...,...,...,...,...,...,...,...,...,...
D12,y,DH5a,pJUMP48-2A(pUC),EZRDM,75ug,1,pJUMP48-2A(pUC)|75ug|1,B2 of Opentrons 6 Tube Rack with Falcon 50 mL ...,H1 of Opentrons 96 Well Aluminum Block with Ge...,D12 of 3 on Temperature Module GEN2 on 3,2022-09-24
E12,y,DH5a,pJUMP48-2A(pUC),EZRDM,75ug,2,pJUMP48-2A(pUC)|75ug|2,B2 of Opentrons 6 Tube Rack with Falcon 50 mL ...,H1 of Opentrons 96 Well Aluminum Block with Ge...,E12 of 3 on Temperature Module GEN2 on 3,2022-09-24
F12,y,DH5a,pJUMP48-2A(pUC),EZRDM,100ug,0,pJUMP48-2A(pUC)|100ug|0,A3 of Opentrons 6 Tube Rack with Falcon 50 mL ...,H1 of Opentrons 96 Well Aluminum Block with Ge...,F12 of 3 on Temperature Module GEN2 on 3,2022-09-24


## Visual Deck Inspection

In [40]:
complete_run_df.to_csv("220925_decryption_spreadsheet.csv")

In [36]:
on_cultures_rack_obj_styler = on_cultures_rack_obj.layout.style.set_table_attributes("style='display:inline'").set_caption(f"Layout of {on_cultures_rack_obj.name} ({resusp_colony_plate.parent})")
media_rack_styler = media_rack_obj.layout.style.set_table_attributes("style='display:inline'").set_caption(f"Layout of {media_rack_obj.name} ({large_vol_rack.parent})")
culture_plate_media_styler = culture_plate_media_obj.layout.style.set_table_attributes("style='display:inline'").set_caption(f"Layout of {culture_plate_media_obj.name} ({culture_plate.parent})")
culture_plate_styler = culture_plate_obj.layout.style.set_table_attributes("style='display:inline'").set_caption(f"Layout of {culture_plate_obj.name} ({culture_plate.parent})")
space = "\xa0"*5
display_html(media_rack_styler._repr_html_()+space+on_cultures_rack_obj_styler._repr_html_()+space+culture_plate_media_styler._repr_html_()+space+culture_plate_styler._repr_html_(), raw=True)
print(f"\n{'-'*100}\n")

Layout of media in 50ml Falkon tubes (4) 
 
 
   
 1 
 2 
 3 
 
 
 
 
 A 
 0ug 
 50ug 
 100ug 
 
 
 B 
 25ug 
 75ug 
 0 
 
 
 
     
 
 Layout of O/N cultures aliquoted in 8-strip tubes (2) 
 
 
   
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 
 
 
 
 A 
 NTC 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 B 
 just_cells 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 C 
 LVL2_NTC 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 D 
 pJUMP47-2A(pSC101) 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 E 
 pJUMP43-2A(pBBR1) 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 F 
 pJUMP46-2A(p15A) 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 G 
 pJUMP44-2A(pRO1600-ColE1) 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 H 
 pJUMP48-2A(pUC) 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
     
 
 Layout of cell culture plate (abx concentrations) (Temperature Module GEN2 on 3) 
 
 
   
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 
 
 
 
 A 
 0ug 
 0ug 
 75ug 
 25ug 
 75ug 
 25ug 
 100ug 
 25ug 
 100ug 
 50ug 
 100ug 
 50ug 
 
 
 B 
 0ug 
 25ug 
 75ug 
 25ug 
 100ug 
 25ug 
 100ug 
 50ug 
 100ug 
 50ug 
 0ug 
 50ug 
 
 
 C 
 0ug 
 25ug 
 100ug 
 25ug 
 100ug 
 50ug 
 100ug 
 50ug 
 0ug 
 50ug 
 0ug 
 75ug 
 
 
 D 
 0ug 
 25ug 
 100ug 
 50ug 
 100ug 
 50ug 
 0ug 
 50ug 
 0ug 
 75ug 
 0ug 
 75ug 
 
 
 E 
 0ug 
 50ug 
 100ug 
 50ug 
 0ug 
 50ug 
 0ug 
 75ug 
 0ug 
 75ug 
 25ug 
 75ug 
 
 
 F 
 0ug 
 50ug 
 0ug 
 50ug 
 0ug 
 75ug 
 0ug 
 75ug 
 25ug 
 75ug 
 25ug 
 100ug 
 
 
 G 
 0ug 
 50ug 
 0ug 
 75ug 
 0ug 
 75ug 
 25ug 
 75ug 
 25ug 
 100ug 
 25ug 
 100ug 
 
 
 H 
 0ug 
 75ug 
 0ug 
 75ug 
 25ug 
 75ug 
 25ug 
 100ug 
 25ug 
 100ug 
 50ug 
 100ug 
 
 
 
     
 
 Layout of cell culture plate (ID) (Temperature Module GEN2 on 3) 
 
 
   
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 
 
 
 
 A 
 NTC|0ug|0 
 LVL2_NTC|0ug|2 
 LVL2_NTC|75ug|1 
 pJUMP47-2A(pSC101)|25ug|0 
 pJUMP47-2A(pSC101)|75ug|2 
 pJUMP43-2A(pBBR1)|25ug|1 
 pJUMP43-2A(pBBR1)|100ug|0 
 pJUMP46-2A(p15A)|25ug|2 
 pJUMP46-2A(p15A)|100ug|1 
 pJUMP44-2A(pRO1600-ColE1)|50ug|0 
 pJUMP44-2A(pRO1600-ColE1)|100ug|2 
 pJUMP48-2A(pUC)|50ug|1 
 
 
 B 
 NTC|0ug|1 
 LVL2_NTC|25ug|0 
 LVL2_NTC|75ug|2 
 pJUMP47-2A(pSC101)|25ug|1 
 pJUMP47-2A(pSC101)|100ug|0 
 pJUMP43-2A(pBBR1)|25ug|2 
 pJUMP43-2A(pBBR1)|100ug|1 
 pJUMP46-2A(p15A)|50ug|0 
 pJUMP46-2A(p15A)|100ug|2 
 pJUMP44-2A(pRO1600-ColE1)|50ug|1 
 pJUMP48-2A(pUC)|0ug|0 
 pJUMP48-2A(pUC)|50ug|2 
 
 
 C 
 NTC|0ug|2 
 LVL2_NTC|25ug|1 
 LVL2_NTC|100ug|0 
 pJUMP47-2A(pSC101)|25ug|2 
 pJUMP47-2A(pSC101)|100ug|1 
 pJUMP43-2A(pBBR1)|50ug|0 
 pJUMP43-2A(pBBR1)|100ug|2 
 pJUMP46-2A(p15A)|50ug|1 
 pJUMP44-2A(pRO1600-ColE1)|0ug|0 
 pJUMP44-2A(pRO1600-ColE1)|50ug|2 
 pJUMP48-2A(pUC)|0ug|1 
 pJUMP48-2A(pUC)|75ug|0 
 
 
 D 
 just_cells|0ug|0 
 LVL2_NTC|25ug|2 
 LVL2_NTC|100ug|1 
 pJUMP47-2A(pSC101)|50ug|0 
 pJUMP47-2A(pSC101)|100ug|2 
 pJUMP43-2A(pBBR1)|50ug|1 
 pJUMP46-2A(p15A)|0ug|0 
 pJUMP46-2A(p15A)|50ug|2 
 pJUMP44-2A(pRO1600-ColE1)|0ug|1 
 pJUMP44-2A(pRO1600-ColE1)|75ug|0 
 pJUMP48-2A(pUC)|0ug|2 
 pJUMP48-2A(pUC)|75ug|1 
 
 
 E 
 just_cells|0ug|1 
 LVL2_NTC|50ug|0 
 LVL2_NTC|100ug|2 
 pJUMP47-2A(pSC101)|50ug|1 
 pJUMP43-2A(pBBR1)|0ug|0 
 pJUMP43-2A(pBBR1)|50ug|2 
 pJUMP46-2A(p15A)|0ug|1 
 pJUMP46-2A(p15A)|75ug|0 
 pJUMP44-2A(pRO1600-ColE1)|0ug|2 
 pJUMP44-2A(pRO1600-ColE1)|75ug|1 
 pJUMP48-2A(pUC)|25ug|0 
 pJUMP48-2A(pUC)|75ug|2 
 
 
 F 
 just_cells|0ug|2 
 LVL2_NTC|50ug|1 
 pJUMP47-2A(pSC101)|0ug|0 
 pJUMP47-2A(pSC101)|50ug|2 
 pJUMP43-2A(pBBR1)|0ug|1 
 pJUMP43-2A(pBBR1)|75ug|0 
 pJUMP46-2A(p15A)|0ug|2 
 pJUMP46-2A(p15A)|75ug|1 
 pJUMP44-2A(pRO1600-ColE1)|25ug|0 
 pJUMP44-2A(pRO1600-ColE1)|75ug|2 
 pJUMP48-2A(pUC)|25ug|1 
 pJUMP48-2A(pUC)|100ug|0 
 
 
 G 
 LVL2_NTC|0ug|0 
 LVL2_NTC|50ug|2 
 pJUMP47-2A(pSC101)|0ug|1 
 pJUMP47-2A(pSC101)|75ug|0 
 pJUMP43-2A(pBBR1)|0ug|2 
 pJUMP43-2A(pBBR1)|75ug|1 
 pJUMP46-2A(p15A)|25ug|0 
 pJUMP46-2A(p15A)|75ug|2 
 pJUMP44-2A(pRO1600-ColE1)|25ug|1 
 pJUMP44-2A(pRO1600-ColE1)|100ug


----------------------------------------------------------------------------------------------------



## Media Preparation

In [37]:
print(f"This experiment requires...")
for unique_abx_c in complete_run_df.abx_c.unique():
    sample_no=len(complete_run_df.loc[complete_run_df.abx_c==unique_abx_c])
    print(f" - {sample_no} samples of {unique_abx_c} ug/ml Streptomycin")
    if sample_no*cell_culture_vol < 5_000:
        print(sample_no*cell_culture_vol)

This experiment requires...
 - 24 samples of 0ug ug/ml Streptomycin
4800.0
 - 18 samples of 25ug ug/ml Streptomycin
3600.0
 - 18 samples of 50ug ug/ml Streptomycin
3600.0
 - 18 samples of 75ug ug/ml Streptomycin
3600.0
 - 18 samples of 100ug ug/ml Streptomycin
3600.0


## Cell Preparation

In [21]:
print(f"This experiment requires...")
for unique_samples in complete_run_df.samples.unique():
    sample_no=len(complete_run_df.loc[complete_run_df.samples==unique_samples])
    print(f" - {sample_no} samples of {unique_samples}")
    print(sample_no*resusp_col_for_cell_culture_vol)

This experiment requires...
 - 6 samples of NTC
6.0
 - 15 samples of LVL2_NTC
15.0
 - 15 samples of pJUMP47-2A(pSC101)
15.0
 - 15 samples of pJUMP43-2A(pBBR1)
15.0
 - 15 samples of pJUMP46-2A(p15A)
15.0
 - 15 samples of pJUMP44-2A(pRO1600-ColE1)
15.0
 - 15 samples of pJUMP48-2A(pUC)
15.0


## Summary Spreadsheets

In [22]:
#pd.DataFrame.from_dict(culture_plate_obj.occupied_wells,orient='index')
complete_run_df#.to_csv(f"_decryption_spreadsheet.csv")

,cells,host,samples,media,abx_c,replicate,ID,media_source,sample_source,destination,prep_date
well,,,,,,,,,,,
A1,n,nan,NTC,EZRDM,0ug,0,NTC|0ug|0,A1 of Opentrons 6 Tube Rack with Falcon 50 mL ...,A1 of Opentrons 96 Well Aluminum Block with Ge...,A1 of 3 on Temperature Module GEN2 on 3,2022-09-24
B1,n,nan,NTC,EZRDM,0ug,1,NTC|0ug|1,A1 of Opentrons 6 Tube Rack with Falcon 50 mL ...,A1 of Opentrons 96 Well Aluminum Block with Ge...,B1 of 3 on Temperature Module GEN2 on 3,2022-09-24
C1,n,nan,NTC,EZRDM,0ug,2,NTC|0ug|2,A1 of Opentrons 6 Tube Rack with Falcon 50 mL ...,A1 of Opentrons 96 Well Aluminum Block with Ge...,C1 of 3 on Temperature Module GEN2 on 3,2022-09-24
D1,n,nan,NTC,EZRDM,100ug,0,NTC|100ug|0,A3 of Opentrons 6 Tube Rack with Falcon 50 mL ...,A1 of Opentrons 96 Well Aluminum Block with Ge...,D1 of 3 on Temperature Module GEN2 on 3,2022-09-24
E1,n,nan,NTC,EZRDM,100ug,1,NTC|100ug|1,A3 of Opentrons 6 Tube Rack with Falcon 50 mL ...,A1 of Opentrons 96 Well Aluminum Block with Ge...,E1 of 3 on Temperature Module GEN2 on 3,2022-09-24
...,...,...,...,...,...,...,...,...,...,...,...
D12,y,DH5a,pJUMP48-2A(pUC),EZRDM,75ug,1,pJUMP48-2A(pUC)|75ug|1,B2 of Opentrons 6 Tube Rack with Falcon 50 mL ...,G1 of Opentrons 96 Well Aluminum Block with Ge...,D12 of 3 on Temperature Module GEN2 on 3,2022-09-24
E12,y,DH5a,pJUMP48-2A(pUC),EZRDM,75ug,2,pJUMP48-2A(pUC)|75ug|2,B2 of Opentrons 6 Tube Rack with Falcon 50 mL ...,G1 of Opentrons 96 Well Aluminum Block with Ge...,E12 of 3 on Temperature Module GEN2 on 3,2022-09-24
F12,y,DH5a,pJUMP48-2A(pUC),EZRDM,100ug,0,pJUMP48-2A(pUC)|100ug|0,A3 of Opentrons 6 Tube Rack with Falcon 50 mL ...,G1 of Opentrons 96 Well Aluminum Block with Ge...,F12 of 3 on Temperature Module GEN2 on 3,2022-09-24


In [23]:
temp_mod.set_temperature(6)
if audiofeedback == True:
    allDone()

---
# 6- Execution


## i. Media Distribution

In [24]:
print(f"Distributing medium...")
for unique_medium in complete_run_df.media_source.unique():
    current_medium_info_df = complete_run_df.loc[complete_run_df.media_source==unique_medium]
    current_medium_destinations_list = current_medium_info_df.destination.to_list()
    print(f" - {cell_culture_vol} ul {current_medium_info_df.abx_c.iloc[0]} Streptomycin\n                -> culture plate {[pos.well_name for pos in current_medium_destinations_list]}")
    smart_distribute_liquid(cell_culture_vol, unique_medium, current_medium_destinations_list,
                            asp_bot_clearance=2,dispens_bot_clearance=2.,
                            asp_flow_rate = 1,disp_flow_rate = 1, new_disposal_volume=0,
                            touch_tip_bool=False,blow_out_bool=False)

Distributing medium...
 - 200.0 ul 0ug Streptomycin
                -> culture plate ['A1', 'B1', 'C1', 'G1', 'H1', 'A2', 'F3', 'G3', 'H3', 'E5', 'F5', 'G5', 'D7', 'E7', 'F7', 'C9', 'D9', 'E9', 'B11', 'C11', 'D11']
 - 200.0 ul 100ug Streptomycin
                -> culture plate ['D1', 'E1', 'F1', 'C3', 'D3', 'E3', 'B5', 'C5', 'D5', 'A7', 'B7', 'C7', 'H8', 'A9', 'B9', 'G10', 'H10', 'A11', 'F12', 'G12', 'H12']
 - 200.0 ul 25ug Streptomycin
                -> culture plate ['B2', 'C2', 'D2', 'A4', 'B4', 'C4', 'H5', 'A6', 'B6', 'G7', 'H7', 'A8', 'F9', 'G9', 'H9', 'E11', 'F11', 'G11']
 - 200.0 ul 50ug Streptomycin
                -> culture plate ['E2', 'F2', 'G2', 'D4', 'E4', 'F4', 'C6', 'D6', 'E6', 'B8', 'C8', 'D8', 'A10', 'B10', 'C10', 'H11', 'A12', 'B12']
 - 200.0 ul 75ug Streptomycin
                -> culture plate ['H2', 'A3', 'B3', 'G4', 'H4', 'A5', 'F6', 'G6', 'H6', 'E8', 'F8', 'G8', 'D10', 'E10', 'F10', 'C12', 'D12', 'E12']


## ii. Cell Distribution

In [25]:
print(f"Distributing samples...")
for unique_sample in complete_run_df.sample_source.unique():
    current_sample_info_df = complete_run_df.loc[complete_run_df.sample_source==unique_sample]
    current_sample_destinations_list = current_sample_info_df.destination.to_list()
    print(f" - {resusp_col_for_cell_culture_vol} ul {current_sample_info_df.samples.iloc[0]} ({unique_sample.well_name})\n          -> culture plate {[pos.well_name for pos in current_sample_destinations_list]}")
    smart_distribute_liquid(resusp_col_for_cell_culture_vol, unique_sample, current_sample_destinations_list,
                            asp_bot_clearance=2,dispens_bot_clearance=2.,
                            asp_flow_rate = 0.9,disp_flow_rate = 0.9, new_disposal_volume=0,
                            touch_tip_bool=False,blow_out_bool=False)

Distributing samples...
 - 1.0 ul NTC (A1)
          -> culture plate ['A1', 'B1', 'C1', 'D1', 'E1', 'F1']
 - 1.0 ul LVL2_NTC (B1)
          -> culture plate ['G1', 'H1', 'A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'A3', 'B3', 'C3', 'D3', 'E3']
 - 1.0 ul pJUMP47-2A(pSC101) (C1)
          -> culture plate ['F3', 'G3', 'H3', 'A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'A5', 'B5', 'C5', 'D5']
 - 1.0 ul pJUMP43-2A(pBBR1) (D1)
          -> culture plate ['E5', 'F5', 'G5', 'H5', 'A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'A7', 'B7', 'C7']
 - 1.0 ul pJUMP46-2A(p15A) (E1)
          -> culture plate ['D7', 'E7', 'F7', 'G7', 'H7', 'A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'A9', 'B9']
 - 1.0 ul pJUMP44-2A(pRO1600-ColE1) (F1)
          -> culture plate ['C9', 'D9', 'E9', 'F9', 'G9', 'H9', 'A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10', 'A11']
 - 1.0 ul pJUMP48-2A(pUC) (G1)
          -> culture plate ['B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11', 'A12', 'B12', 'C12', '

# Shutdown

In [26]:
# allDone()
try:
    p20_left.drop_tip()
except:
    pass
try:
    p300_right.drop_tip()
except:
    pass
temp_mod.deactivate()
protocol.home()
#os.system("systemctl start opentrons-robot-server")

In [27]:
# complete tip usage check:
_=[display(show_rack_usage(labware)[1]) for labware in labware_list if 'Opentrons 96 Tip Rack' in str(labware)]

Opentrons 96 Tip Rack 20 µL on 11:
 -> 8 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,x,0,0,0,0,0,0,0,0,0,0,0
B,x,0,0,0,0,0,0,0,0,0,0,0
C,x,0,0,0,0,0,0,0,0,0,0,0
D,x,0,0,0,0,0,0,0,0,0,0,0
E,x,0,0,0,0,0,0,0,0,0,0,0
F,x,0,0,0,0,0,0,0,0,0,0,0
G,x,0,0,0,0,0,0,0,0,0,0,0
H,x,0,0,0,0,0,0,0,0,0,0,0


Opentrons 96 Tip Rack 20 µL on 10:
 -> 0 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,0,0,0,0,0,0,0,0,0,0,0,0
B,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0
D,0,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0,0


Opentrons 96 Tip Rack 300 µL on 7:
 -> 5 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,x,0,0,0,0,0,0,0,0,0,0,0
B,x,0,0,0,0,0,0,0,0,0,0,0
C,x,0,0,0,0,0,0,0,0,0,0,0
D,x,0,0,0,0,0,0,0,0,0,0,0
E,x,0,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0,0
